In [13]:
import datasets
from transformers import AutoModelForSequenceClassification, AutoTokenizer,Trainer,TrainingArguments,DataCollatorWithPadding
import pandas 
import torch
from evaluate import load
modelName="microsoft/graphcodebert-base"
model=AutoModelForSequenceClassification.from_pretrained(modelName,num_labels=4)
tokenizer=AutoTokenizer.from_pretrained(modelName)
data_collator=DataCollatorWithPadding(tokenizer)

# for param in model.base_model.parameters():
#     param.requires_grad = False

# for param in model.classifier.parameters():
#     param.requires_grad = True

print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: 124648708


In [4]:

trainds=datasets.load_dataset("parquet",data_files="task_c/task_c_training_set_1.parquet")["train"]
testds=datasets.load_dataset("parquet",data_files="task_c/task_c_test_set_sample.parquet")["train"]
valds=datasets.load_dataset("parquet",data_files="task_c/task_c_validation_set.parquet")["train"]


In [5]:
from collections import Counter
Counter(trainds['label'])


Counter({1: 210471, 0: 485483, 2: 85520, 3: 118526})

In [6]:

collection=[]
for i in range(4):
    classDs=trainds.filter(lambda x: x["label"]==i)
    classDs=classDs.shuffle(seed=36)
    classDs=classDs.select(range(85520))
    collection.append(classDs)

downsampledDS=datasets.concatenate_datasets(collection)
trainds=downsampledDS.shuffle(seed=42)


In [7]:
def tokenFunction(row):
    text = f"{row['code']} </s> {row['generator']} </s> {row['language']}"
    tokens= tokenizer(text,truncation=True, padding="max_length", max_length=256)
    tokens["labels"]=row["label"]
    return tokens
tokenizedtrain=trainds.map(tokenFunction)
tokenizedtest=testds.map(tokenFunction)
tokenizedval=valds.map(tokenFunction)


Map:   0%|          | 0/342080 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200000 [00:00<?, ? examples/s]

In [14]:
trainingArgs=TrainingArguments(
    output_dir="./results/SemEvalC",
    eval_strategy="epoch",
    fp16=True,
    num_train_epochs=4,
    learning_rate=1e-5,
    per_device_eval_batch_size=8,
    logging_steps=10000,
    save_strategy="epoch",
)

f1_func=load("f1")
acc_func=load("accuracy")
recall_func=load("recall")
def f1_clac(preds):
    logits=preds.predictions
    labels=preds.label_ids
    prediction=logits.argmax(axis=-1)
    return {
        "accuracy":acc_func.compute(predictions=prediction,references=labels)['accuracy'],
        "recall":recall_func.compute(predictions=prediction,references=labels, average="macro")['recall'],
        "f1":f1_func.compute(predictions=prediction,references=labels,average="macro")['f1'],
    }

trainer =Trainer(
    model=model,
    args=trainingArgs,
    tokenizer=tokenizer,
    compute_metrics=f1_clac,
    train_dataset=tokenizedtrain,
    eval_dataset=tokenizedval,
    data_collator=data_collator
)
trainer.train()
test_metrics = trainer.evaluate(tokenizedtest)
trainer.save_model("./results/SemEvalC/final_model_graphbert")

# Save the tokenizer
tokenizer.save_pretrained("./results/SemEvalC/final_model_graphbert")
print(f"Test Loss: {test_metrics['eval_loss']:.4f}, F1: {test_metrics['eval_f1']:.4f}")

C:\Users\purrpl\AppData\Local\Temp\ipykernel_28272\1053581023.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer =Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Recall,F1
1,0.450700,0.365631,0.860315,0.825974,0.796721
2,0.394300,0.390883,0.864550,0.836490,0.804109
3,0.343700,0.400698,0.869300,0.836390,0.809063
4,0.304800,0.462213,0.872725,0.838372,0.812970


Test Loss: 0.4357, F1: 0.8181
